# Script for identifying marine heatwaves (MHWs) in the Barents Sea

#### First, as we are using Python we need to import some libraries. **To quickly run a cell :"Ctrl+Enter"**. Try it with the cell bellow to import the libraries.

In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.patches as mpatches
import matplotlib.dates as mdates
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import glob
import datetime
from datetime import date
import pandas as pd
import csv

#### To avoid warning messages run the cell below:

In [ ]:
import warnings
warnings.filterwarnings('ignore')

#### We also need a statistical tool to remove the trend. Import this by running the cell below:

In [ ]:
from scipy import signal

#### Next import the **marineHeatWaves** module – conda must be run from /Python/MarineHeatWaves/directory


In [ ]:
import marineHeatWaves as mhw

#### Add in the paths where you have the data saved `dat_dir` and where you would like the figures to be saved `fig_dir`

In [ ]:
dat_dir = ''
sav_dir = dat_dir
fig_dir = ''

#### Choose which region you would like to focus on: 
> (uncomment the region you would like to look at)

In [ ]:
#area = 'BIT'   # Bear Island Trough / BSO
#area = 'PS'    # Pechora Sea
#area = 'SB'    # Spitsbergen Bank
#area = 'NB'    # North-eastern Barents Sea / BSX
area = 'Full'   # Full Barents Sea

#### Choose whether you would like to focus on surface or bottom MHWs:
- `surfbot = 1`: surface  
- `surfbot = 2`: bottom

In [ ]:
surfbot = 2

#### Load the data

In [ ]:
if surfbot == 1:
   ds = xr.open_dataset(dat_dir + 'TOPAZ_Tmp_Sur_' + area + '_1991_2022.nc')
elif surfbot == 2:
   ds = xr.open_dataset(dat_dir + 'TOPAZ_Tmp_Bot_' + area + '_1991_2022.nc')

#### Average the data spatially

In [ ]:
if surfbot == 1:
   ds_avg = ds['thetao'].mean(dim=["longitude","latitude","depth"],skipna=True).squeeze()
elif surfbot == 2:
   ds_avg = ds['bottomT'].mean(dim=["longitude","latitude"],skipna=True).squeeze()

#### Put the time values into vector 'date'
> This makes the dates appear in the format '2021-06-01'



In [ ]:
ocean_date = ds['time'].data.astype('datetime64[D]')

#### Get your average temperature variable 

In [ ]:
temp = ds_avg.data

#### Convert the date from np-datetime64 to datetime.date()
> Dates become a list of Python datetime.date objects eg., 'datetime.date(2021, 6, 1)'

In [ ]:
date_datetime = ocean_date.tolist()
t = np.asarray([datetime.date.toordinal(tt) for tt in date_datetime])
#t = np.arange(date(1991,1,1).toordinal(),date(2021,12,31).toordinal()+1)
dates = [date.fromordinal(tt.astype(int)) for tt in t]

### Great! Now that you are set up you can use the MHW functions to detect MHWs and get statistics.
- `mhw.detect()` outputs a list of MHW events over our time period and their characteristics, including **frequency** (number of events per year), **intensity** (maximum sea surface temperature anomaly, °C), and **duration** (days).  
- It also outputs a climatology relative to which the MHW is defined.


#### Choose which climatological reference period you would like to use:

- `1` = 1991–2020  
- `2` = 1996–2020  
- `3` = 2001–2020


In [ ]:
clm = 1
if clm == 1:
   mhws, clim = mhw.detect(t, temp, [1991,2020])   # Using 1991-2020 for climatology
   mhwc, climc = mhw.detect(t, temp, [1991,2020], coldSpells = True)   # Cold spells, Using 1991-2020 for climatology
elif clm == 2:
   mhws, clim = mhw.detect(t, temp, [1996,2020])   # Using 1996-2020 for climatology
elif clm == 3:
   mhws, clim = mhw.detect(t, temp, [2001,2020])   # Using 2001-2020 for climatology

#### Calculate statistics using `blockAverage`
> Use this function to compute averages for the selected time period.


In [ ]:
mhwBlock = mhw.blockAverage(t, mhws)

#### Print results to a csv-file:

In [ ]:
print_file = 0
if print_file == 1:
   if surfbot == 1:
      f = open(dat_dir + 'MHW_statistics_' + area + '_surface.csv', 'w')
   elif surfbot == 2:
      f = open(dat_dir + 'MHW_statistics_' + area + '_bottom.csv', 'w')
   writer = csv.writer(f)

#writer.writeheader(["daynumber", "clim season", "90 pct", "temp"])
   for i in range(0, len(t)):
      writer.writerow([t[i], clim['seas'][i], clim['thresh'][i], temp[i]])

   f.close()

#### Print Statistics:

In [ ]:
if surfbot == 1:
   if clm == 1:
      print("*** Surface statistics from TOPAZ for area " + area + " using climate period 1991-2020 ***")
   elif clm == 2:
      print("*** Surface statistics from TOPAZ for area " + area + " using climate period 1996-2020 ***")
   elif clm == 3:
      print("*** Surface statistics from TOPAZ for area " + area + " using climate period 2001-2020 ***")
elif surfbot == 2:
   if clm == 1:
      print("*** Bottom statistics from TOPAZ for area " + area + " using climate period 1991-2020 ***")
   elif clm == 2:
      print("*** Bottom statistics from TOPAZ for area " + area + " using climate period 1996-2020 ***")
   elif clm == 3:
      print("*** Bottom statistics from TOPAZ for area " + area + " using climate period 2001-2020 ***")

mean, trend, dtrend = mhw.meanTrend(mhwBlock)

#### Print the characteristics of the events (frequency, intensity and duration):

In [ ]:
print_stat = 0
if print_stat == 1:
 
   print("There are on average " + str(mean['count']) + " MHWs in each year, \n \
   with a linear trend of " + str(10*trend['count']) + " MHW events per decade \n \
   This trend is statistically significant (p<0.05): " \
   + str(np.abs(trend['count']) > dtrend['count']) + "\n")

   print("The average maximum intensity is " + str(mean['intensity_max']) + " deg C above climatology, \n \
   with a linear trend of " + str(10*trend['intensity_max']) + " deg. C per decade \n \
   This trend is statistically significant (p<0.05): " \
   + str(np.abs(trend['intensity_max']) > dtrend['intensity_max']) + "\n")

   print("The average duration is " + str(mean['duration']) + " days per year, \n \
   with a linear trend of " + str(10*trend['duration']) + " days per year per decade \n \
   This trend is statistically significant (p<0.05): " \
   + str(np.abs(trend['duration']) > dtrend['duration']) + "\n")

#### Check miscellaneous properties of MHWs for validation:

In [ ]:
misc = 0
if misc == 1:
   ev = np.argmax(mhws['intensity_cumulative'])

   print('** Statistics on most intense event **')
   print('Maximum intensity:', mhws['intensity_max'][ev], 'deg. C above the climatology')
   print('Average intensity:', mhws['intensity_mean'][ev], 'deg. C above the climatology')
   print('Maximum duration:', mhws['duration'][ev], 'days')
   print('Start date:', mhws['date_start'][ev].strftime("%d %B %Y"))
   print('End date:', mhws['date_end'][ev].strftime("%d %B %Y"))

## Plotting

#### Plot the event with the maximum cumulative intensity:

In [ ]:
   fig, axs = plt.subplots(1, 1, figsize=(14,5))
   plt.rc('font', size=14)
   plt.rc('axes', labelsize=20)
   plt.rc('xtick', labelsize=20)
   plt.rc('ytick', labelsize=20)
   plt.xticks(fontsize = 14)
   plt.yticks(fontsize = 14)
# Find indices for maximum cumulative event and shade all XX MHWs before and after in pink:
#   for ev0 in np.arange(ev-29, ev+8, 1):
# Spans used for Figure 3 (DO NOT CHANGE!):
#   for ev0 in np.arange(ev-10, ev+10, 1):  # Full Surface
#   for ev0 in np.arange(ev-5, ev+1, 1):    # Full Bottom
#   for ev0 in np.arange(ev-12, ev+20, 1):  # BIT Surface
#   for ev0 in np.arange(ev-6, ev+3, 1):    # BIT Bottom
#   for ev0 in np.arange(ev-30, ev+2, 1):   # NB Surface
   for ev0 in np.arange(ev-5, ev+1, 1):    # NB Bottom
#   for ev0 in np.arange(ev-12, ev+6, 1):   # SB Surface
#   for ev0 in np.arange(ev-10, ev+4, 1):   # SB Bottom
#   for ev0 in np.arange(ev-12, ev+12, 1):  # PS Surface
#   for ev0 in np.arange(ev-6, ev+3, 1):    # PS Bottom
# Spans used for Supplementary Figure (DO NOT CHANGE!):
#   for ev0 in np.arange(ev-18, ev+2, 1):  # Full Surface 1 (1991 - 2005)
#   for ev0 in np.arange(ev-1, ev+1, 1):    # Full Bottom 1 (1991 - 2005)
#   for ev0 in np.arange(ev-16, ev+12, 1):  # Full Surface 2 (2005 - 2022)
#   for ev0 in np.arange(ev-4, ev+1, 1):    # Full Bottom 2 (2005 - 2022)
# Spans used for 2012 Check:
#   for ev0 in np.arange(ev-12, ev+2, 1):  # Full Surface (2012 - 2014)
#   for ev0 in np.arange(ev-4, ev+1, 1):    # Full Bottom (2012 - 2014)
#   for ev0 in np.arange(ev-16, ev+12, 1):  # BIT Surface (2012 - 2014)
#   for ev0 in np.arange(ev-10, ev+1, 1):    # BIT Bottom (2012 - 2014)
#   for ev0 in np.arange(ev-16, ev+12, 1):  # PS Surface (2012 - 2014)
#   for ev0 in np.arange(ev-8, ev+1, 1):    # PS Bottom (2012 - 2014)
#   for ev0 in np.arange(ev-18, ev+6, 1):  # NB Surface (2012 - 2014)
#   for ev0 in np.arange(ev-6, ev+1, 1):    # NB Bottom (2012 - 2014)
      t1 = np.where(t==mhws['time_start'][ev0])[0][0]
      t2 = np.where(t==mhws['time_end'][ev0])[0][0]
      plt.fill_between(dates[t1:t2+1], temp[t1:t2+1], clim['thresh'][t1:t2+1], color=(1,0.6,0.5))
# Shade largest cumulative event in dark red:
   t1 = np.where(t==mhws['time_start'][ev])[0][0]
   t2 = np.where(t==mhws['time_end'][ev])[0][0]
   plt.fill_between(dates[t1:t2+1], temp[t1:t2+1], clim['thresh'][t1:t2+1], color='r')
# Plot SST, seasonal cycle, threshold, shade MHWs with main event in red
   axs.plot(dates, temp, 'k-', linewidth=2)
   axs.plot(dates, clim['thresh'], 'g-', linewidth=2)
   axs.plot(dates, clim['seas'], 'b-', linewidth=2)
   if surfbot == 1:
      axs.set_xlim(16436, 17532)   # 1.1.2015 - 1.1.2018
#      axs.set_xlim(15340, 16071)   # 1.1.2012 - 1.1.2014
#      axs.set_xlim(7670, 19358)   # 1.1.1991 - 1.1.2023
#      axs.set_xlim(7670, 12784)   # 1.1.1991 - 1.1.2005
#      axs.set_xlim(12784, 19358)   # 1.1.2005 - 1.1.2023
   elif surfbot == 2:
      axs.set_xlim(16436, 17532)   # 1.1.2015 - 1.1.2018
#      axs.set_xlim(15340, 16071)   # 1.1.2012 - 1.1.2014
#      axs.set_xlim(7670, 19358)   # 1.1.1991 - 1.1.2023
#      axs.set_xlim(7670, 12784)   # 1.1.1991 - 1.1.2005
#      axs.set_xlim(12784, 19358)   # 1.1.2005 - 1.1.2023
      axs.set_ylim(clim['seas'].min() - 1, clim['seas'].max() + mhws['intensity_max'][ev] + 0.5)
   if surfbot == 1:
      axs.set_ylabel(r'Surface temperature [$^\circ$C]', size=14)
   elif surfbot == 2:
      axs.set_ylabel(r'Bottom temperature [$^\circ$C]', size=14)
#   axs.xaxis.set_major_locator(mdates.MonthLocator(bymonth=(1, 4, 7, 10)))
   axs.xaxis.set_major_locator(mdates.MonthLocator(bymonth=(1, 7)))   # Used in Fig. 3 (DO NOT CHANGE!)
#   axs.xaxis.set_minor_locator(mdates.MonthLocator())
#   axs.xaxis.set_major_locator(mdates.YearLocator(base=5))
#   axs.xaxis.set_minor_locator(mdates.YearLocator())
#   axs.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%b'))
# Inserting panel labels:
   if surfbot == 1:
      if area == 'Full':
         plt.text(.05,.9, 'a)', fontsize = 14, transform = axs.transAxes)
#         plt.text(.05,.9, 'c)', fontsize = 14, transform = axs.transAxes)
      elif area == 'BIT':
         plt.text(.05,.9, 'b)', fontsize = 14, transform = axs.transAxes)
      elif area == 'NB':
         plt.text(.05,.9, 'c)', fontsize = 14, transform = axs.transAxes)
      elif area == 'SB':
         plt.text(.05,.9, 'd)', fontsize = 14, transform = axs.transAxes)
      elif area == 'PS':
         plt.text(.05,.9, 'e)', fontsize = 14, transform = axs.transAxes)
   elif surfbot == 2:
      if area == 'Full':
         plt.text(.05,.9, 'f)', fontsize = 14, transform = axs.transAxes)
#         plt.text(.05,.9, 'd)', fontsize = 14, transform = axs.transAxes)
      elif area == 'BIT':
         plt.text(.05,.9, 'g)', fontsize = 14, transform = axs.transAxes)
      elif area == 'NB':
         plt.text(.05,.9, 'h)', fontsize = 14, transform = axs.transAxes)
      elif area == 'SB':
         plt.text(.05,.9, 'i)', fontsize = 14, transform = axs.transAxes)
      elif area == 'PS':
         plt.text(.05,.9, 'j)', fontsize = 14, transform = axs.transAxes)


   plt.show()

#### Plot an Ocean Monitoring Index (OMI) Figure:

> NB an Ocean Monitoring Index plot is a time series visualization used to track and analyze MHWs relative to a defined climatology.

In [ ]:
omi_fig = 1
if omi_fig == 1:
   omi_index = (temp - clim['seas']) / (clim['thresh'] - clim['seas'])

   fig, axs = plt.subplots(1, 1, figsize=(20,4))
   plt.rc('font', size=14)
   plt.rc('axes', labelsize=20)
   plt.rc('xtick', labelsize=20)
   plt.rc('ytick', labelsize=20)
   plt.xticks(fontsize = 14)
   plt.yticks(fontsize = 14)

   ev = np.argmax(mhws['intensity_cumulative'])
   if surfbot == 1:
      for ev0 in np.arange(ev-20, ev+12, 1):
         t1 = np.where(t==mhws['time_start'][ev0])[0][0]
         t2 = np.where(t==mhws['time_end'][ev0])[0][0]
         plt.fill_between(dates[t1:t2+1], omi_index[t1:t2+1], 1, color='r')
   elif surfbot == 2:
      for ev0 in np.arange(ev-4, ev+1, 1):
         t1 = np.where(t==mhws['time_start'][ev0])[0][0]
         t2 = np.where(t==mhws['time_end'][ev0])[0][0]
         plt.fill_between(dates[t1:t2+1], omi_index[t1:t2+1], 1, color='r')

   evc = np.argmax(mhwc['intensity_cumulative'])
   if surfbot == 1:
      for evc0 in np.arange(evc-26, evc+4, 1):
         t1 = np.where(t==mhwc['time_start'][evc0])[0][0]
         t2 = np.where(t==mhwc['time_end'][evc0])[0][0]
         plt.fill_between(dates[t1:t2+1], omi_index[t1:t2+1], -1, color='b')
#         plt.fill_between(dates[t1:t2+1], -1, omi_index[t1:t2+1], color='b')
   elif surfbot == 2:
      for evc0 in np.arange(evc-9, evc+2, 1):
         t1 = np.where(t==mhwc['time_start'][evc0])[0][0]
         t2 = np.where(t==mhwc['time_end'][evc0])[0][0]
#         plt.fill_between(dates[t1:t2+1], omi_index[t1:t2+1], -1, color='b')
         plt.fill_between(dates[t1:t2+1], -1, omi_index[t1:t2+1], color='b')
   axs.plot(dates, omi_index, 'k-', linewidth=2)


   axs.set_xlim(7670, 19358)   # 1.1.1991 - 1.1.2023
   axs.set_ylim(- 2.5, 2.5)

   if surfbot == 1:
      axs.set_ylabel(r'Surface MHW index', size=14)
   elif surfbot == 2:
      axs.set_ylabel(r'Bottom MHW index', size=14)

   axs.xaxis.set_major_locator(mdates.YearLocator(base=5))

   plt.show()

#### Plot a bar chart showing the duration of all events:

In [ ]:
barchart = 0
if barchart == 1:
   ev = np.argmax(mhws['intensity_cumulative'])

   plt.figure(figsize=(14,5))
# Duration
   plt.subplot(1,1,1)
   evMax = np.argmax(mhws['duration'])
   plt.bar(range(mhws['n_events']), mhws['duration'], width=0.6, color=(0.7,0.7,0.7))
   plt.bar(evMax, mhws['duration'][evMax], width=0.6, color=(1,0.5,0.5))
   plt.bar(ev, mhws['duration'][ev], width=0.6, edgecolor=(1,0.,0.), color='none')
   plt.xlim(0, mhws['n_events'])
   plt.ylabel('[days]')
   plt.title('Duration')
   plt.show()


#### Plot timeseries of the number of events (frequency), max intensity, and duration of all events:

In [ ]:
tseries = 0
if tseries == 1:

   plt.figure(figsize=(14,4))

   plt.subplot(2,2,1)
   plt.plot(mhwBlock['years_centre'], mhwBlock['count'], 'k-o')
   plt.ylim(0,9)
   plt.ylabel('[Count]')
   plt.title('Number of MHWs by year')

   plt.subplot(2,2,2)
   plt.plot(mhwBlock['years_centre'], mhwBlock['duration'], 'k-o')
   plt.ylim(0,400)
   plt.ylabel(r'[Days]')
   plt.title('MHW duration')

   plt.subplot(2,2,3)
   plt.plot(mhwBlock['years_centre'], mhwBlock['total_icum'], 'k-o')
   plt.ylabel(r'[$^\circ$C x days]')
#   plt.title('Average MHW maximum intensity by year')

   ev = np.argmax(mhws['intensity_cumulative'])
   evMax = np.argmax(mhws['duration'])
   plt.subplot(2,2,4)
   plt.bar(range(mhws['n_events']), mhws['duration'], width=0.6, color=(0.7,0.7,0.7))
   plt.bar(evMax, mhws['duration'][evMax], width=0.6, color=(1,0.5,0.5))
   plt.bar(ev, mhws['duration'][ev], width=0.6, edgecolor=(1,0.,0.), color='none')
   plt.xlim(0, mhws['n_events'])
   plt.xlabel('[MHW #]')
   plt.ylabel('[Days]')
#   plt.title('Duration')


   plt.show()